In [93]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler,OneHotEncoder,LabelEncoder
from sklearn.model_selection import train_test_split
import pickle


In [94]:
df=pd.read_csv('Churn_Modelling.csv')
df.head(3)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1


In [95]:
df=df.drop(['RowNumber','CustomerId','Surname'],axis=1)

In [96]:
label_encoder_gender=LabelEncoder()

df['Gender']=label_encoder_gender.fit_transform(df['Gender'])
df.head(3)


,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1


In [97]:
one_hot_geo=OneHotEncoder()

df_geography=one_hot_geo.fit_transform(df[['Geography']]).toarray()

df_geography

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [98]:
one_hot_geo.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [99]:
df_geo_encoder=pd.DataFrame(df_geography,columns=one_hot_geo.get_feature_names_out(['Geography']))

In [100]:
df_geo_encoder.head(3)

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0


In [101]:
df_main=pd.concat([df.drop(['Geography'],axis=1),df_geo_encoder],axis=1)

In [102]:
df_main.head(5)

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [103]:
with open('label_encoder_gender.pkl','wb') as file:
    pickle.dump(label_encoder_gender,file)

In [104]:
with open('one_hot_geo.pkl','wb') as file:
    pickle.dump(one_hot_geo,file)

In [105]:
X=df_main.drop('Exited',axis=1)
y=df_main['Exited']

In [106]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=25)

In [107]:
X_train.shape

(7000, 12)

In [108]:
y_train.shape

(7000,)

In [109]:
sc=StandardScaler()
X_train=sc.fit_transform(X_train)
X_test=sc.transform(X_test)

In [110]:
with open('standard Scaler.pkl','wb') as file:
    pickle.dump(sc,file)

In [111]:
X_train

array([[ 0.07251454, -1.07943118,  0.19207892, ..., -1.00286124,
        -0.58064944,  1.74870553],
       [ 0.66375026,  0.92641386, -1.04372295, ...,  0.99714693,
        -0.58064944, -0.57185157],
       [ 0.39406379,  0.92641386, -1.23384632, ...,  0.99714693,
        -0.58064944, -0.57185157],
       ...,
       [-0.19717193,  0.92641386, -0.56841454, ...,  0.99714693,
        -0.58064944, -0.57185157],
       [-0.14530914, -1.07943118, -0.94866127, ..., -1.00286124,
         1.72220953, -0.57185157],
       [-1.037349  , -1.07943118,  0.09701724, ...,  0.99714693,
        -0.58064944, -0.57185157]])

In [112]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Input
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime 

In [113]:
X.shape[1]

12

In [114]:
model=Sequential([
    Input(shape=(X.shape[1],)),
Dense(128,activation='relu'),
Dense(64,activation='relu'),
Dense(32,activation='relu'),
Dense(1,activation='sigmoid')
])

In [115]:
model.summary()

Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_21 (Dense)                │ (None, 128)            │         1,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_22 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_23 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_24 (Dense)                │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 12,033 (47.00 KB)

 Trainable params: 12,033 (47.00 KB)

 Non-trainable params: 0 (0.00 B)

In [116]:
opt=tf.keras.optimizers.Adam(0.01)
loss=tf.keras.losses.BinaryCrossentropy()

In [117]:
model.compile(optimizer=opt,loss=loss,metrics=['accuracy'])

In [118]:

log_dir="logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

In [119]:


tensorflow_callback=TensorBoard(log_dir=log_dir,histogram_freq=1)

earlystopping_callback=EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)

In [120]:
history=model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=100,
                  callbacks=[tensorflow_callback,earlystopping_callback])

Epoch 1/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.8079 - loss: 0.4499 - val_accuracy: 0.8613 - val_loss: 0.3462
Epoch 2/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8571 - loss: 0.3498 - val_accuracy: 0.8640 - val_loss: 0.3435
Epoch 3/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8557 - loss: 0.3449 - val_accuracy: 0.8620 - val_loss: 0.3450
Epoch 4/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8568 - loss: 0.3369 - val_accuracy: 0.8600 - val_loss: 0.3436
Epoch 5/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8655 - loss: 0.3346 - val_accuracy: 0.8607 - val_loss: 0.3432
Epoch 6/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8619 - loss: 0.3350 - val_accuracy: 0.8533 - val_loss: 0.3502
Epoch 7/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8640 - loss: 0.3357 - val_accuracy: 0.8587 - val_loss: 0.3464
Epoch 8/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8595 - loss: 0.3361 - val_accu

In [121]:
model.save('ann_model.h5')

In [122]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [123]:
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6006 (pid 51820), started 0:11:23 ago. (Use '!kill 51820' to kill it.)